# to-dos
- [x] understand the datsets
- [x] write a function to do it for all states
- [ ] calculation for ratio is correct or not in ratioFinder func
- [ ] one state(code=30) has ratio-1(people who speak only one language) to be negative and but still sum is 1
= [ ] now with this understanding I wan to say:
    - first is total pop that speaks that language be it 
    - second is 2+(3+ included)
    - third is about three plus

In [1]:
# pandas basic methods to study dataset #
# head()
# tail()
# shape
# dtype
# astype
# isnull
# dropna
# fillna


In [2]:
#pd.__version__
#pd.io.excel.xlsx['openpyxl']
#pd.set_option("xlsx","openpyxl")
# above both options does not work :
# currently have to specify engine in pd.read_excel itself
# “openpyxl” supports newer Excel file formats
# link:https://stackoverflow.com/questions/60288732/pandas-read-excel-returns-pendingdeprecationwarning
# engine has been specified to deal with warning

In [3]:
import pandas as pd

In [4]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

In [5]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePop=census.iloc[stateCode,5] 
    # modify state code
    stateCode_str=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode_str}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no 
    # note: these col numbers are specific to df #
    total3=sum(df.iloc[:,14].tolist())
    total2=sum(df.iloc[:,9].tolist())-total3 # need to remove total3 as itself is of 2+ language(that also includes 3+ lang persons) speakers
    total1=sum(df.iloc[:,4].tolist())-total2-total3 # as previously I was double substracting 3+ persons I was getting negative number for that
    # doing -toatal3 because total2 has been modified above with toatl3 being substracted
    ratio_1=round(100*total1/statePop,2)
    ratio_2=round(100*total2/statePop,2)
    ratio_3=round(100*total3/statePop,2)
    #store ratios in a dict
    stateRatios={
        'state-code':stateCode,
        'percent-one':ratio_1,
        'percent-two':ratio_2,
        'percent-three':ratio_3,
    }
    #print('Processed state -- ',stateCode)
    return stateRatios


In [6]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)
ratioDF.to_csv('outputs/percent-india.csv',index=False)

CPU times: user 19.4 s, sys: 44.6 ms, total: 19.4 s
Wall time: 19.5 s
